<a href="https://colab.research.google.com/github/Masakinka/python_for_ds_task/blob/main/HW_%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D0%B8_%D0%BF%D0%BE%D0%BD%D0%B8%D0%B6%D0%B5%D0%BD%D0%BD%D1%8F_%D1%80%D0%BE%D0%B7%D0%BC%D1%96%D1%80%D0%BD%D0%BE%D1%81%D1%82%D1%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Домашнє завдання: Пониження розмірностей для Аналізу Портретів Клієнтів

#### Контекст
В цьому ДЗ ми попрацюємо з методами пониження розмірності на наборі даних для задачі аналізу портретів клієнтів (Customer Personality Analysis). **В попередньому ДЗ ми працювали з цими даними використовуючи кластеризацію, зараз використаємо кластеризацію і візуалізауємо результати з різними методами.**

Customer Personality Analysis - це аналіз різних сегментів клієнтів компанії. Цей аналіз дозволяє бізнесу краще розуміти своїх клієнтів і полегшує процес адаптації продуктів під конкретні потреби, поведінку та інтереси різних типів клієнтів.

Аналіз портретів клієнтів допомагає бізнесу змінювати свій продукт на основі цільової аудиторії, розділеної на різні сегменти. Наприклад, замість того, щоб витрачати гроші на маркетинг нового продукту для всіх клієнтів у базі даних компанії, бізнес може проаналізувати, який сегмент клієнтів найімовірніше придбає продукт, і потім зосередити маркетингові зусилля лише на цьому сегменті.

#### Вхідні дані
Вам надано набір даних з такими атрибутами:

**Характеристики користувачів:**
- `ID`: Унікальний ідентифікатор клієнта
- `Year_Birth`: Рік народження клієнта
- `Education`: Рівень освіти клієнта
- `Marital_Status`: Сімейний стан клієнта
- `Income`: Річний дохід домогосподарства клієнта
- `Kidhome`: Кількість дітей у домогосподарстві клієнта
- `Teenhome`: Кількість підлітків у домогосподарстві клієнта
- `Dt_Customer`: Дата реєстрації клієнта у компанії
- `Recency`: Кількість днів з моменту останньої покупки клієнта
- `Complain`: 1, якщо клієнт скаржився за останні 2 роки, 0 - якщо ні

**Продукти:**
- `MntWines`: Сума, витрачена на вино за останні 2 роки
- `MntFruits`: Сума, витрачена на фрукти за останні 2 роки
- `MntMeatProducts`: Сума, витрачена на м'ясні продукти за останні 2 роки
- `MntFishProducts`: Сума, витрачена на рибні продукти за останні 2 роки
- `MntSweetProducts`: Сума, витрачена на солодощі за останні 2 роки
- `MntGoldProds`: Сума, витрачена на золото за останні 2 роки

**Акції:**
- `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
- `AcceptedCmp1`: 1, якщо клієнт прийняв пропозицію у першій кампанії, 0 - якщо ні
- `AcceptedCmp2`: 1, якщо клієнт прийняв пропозицію у другій кампанії, 0 - якщо ні
- `AcceptedCmp3`: 1, якщо клієнт прийняв пропозицію у третій кампанії, 0 - якщо ні
- `AcceptedCmp4`: 1, якщо клієнт прийняв пропозицію у четвертій кампанії, 0 - якщо ні
- `AcceptedCmp5`: 1, якщо клієнт прийняв пропозицію у п'ятій кампанії, 0 - якщо ні
- `Response`: 1, якщо клієнт прийняв пропозицію в останній кампанії, 0 - якщо ні

**Взаємодія з компанією:**
- `NumWebPurchases`: Кількість покупок, зроблених через вебсайт компанії
- `NumCatalogPurchases`: Кількість покупок, зроблених за каталогом
- `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
- `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць


Для початку, запустіть код нижче. Всі ці кроки ми робили в попередньому ДЗ і для того, щоб результати кластеризації у нас були схожими, потрібно аби передобробка була однаковою.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.manifold import TSNE

In [5]:
# 1. Завантаження даних
df = pd.read_csv("drive/MyDrive/Colab Notebooks/data/marketing_campaign.csv", sep='\t')

# 2. Обробка пропущених значень
df['Income_not_filled'] = df.Income.isna()
df.Income = df.Income.fillna(-1)

# 3. Обробка дати реєстрації
df.Dt_Customer = pd.to_datetime(df.Dt_Customer, format='%d-%m-%Y')
today = df.Dt_Customer.max()
df['days_lifetime'] = (today - df.Dt_Customer).dt.days
df['years_customer'] = df.Year_Birth.apply(lambda x: today.year - x)

# 4. Категоризація рівня освіти
df_education = pd.get_dummies(df.Education, prefix='education').astype(int)
df = pd.concat([df, df_education], axis=1)

# 5. Очищення сімейного стану
marital_status_map = {'Alone': 'Single', 'Absurd': 'Else', 'YOLO': 'Else'}
df['Marital_Status_clean'] = df.Marital_Status.map(marital_status_map)
df_ms = pd.get_dummies(df.Marital_Status_clean, prefix='marital').astype(int)
df = pd.concat([df, df_ms], axis=1)

# 6. Форматування доходу і видалення викиду
df.Income = df.Income.astype(int)
df = df[df.Income != 666666]

# 7. Створення фінального набору даних
X = df.drop(['ID', 'Dt_Customer', 'Education', 'Marital_Status', 'Marital_Status_clean'], axis=1)
X.reset_index(drop=True, inplace=True)

### Завдання 1: Виконання кластеризації та пониження розмірності для візуалізації результатів

Ваше завдання — провести кластеризацію клієнтів та візуалізувати результати кластеризації, використовуючи метод головних компонент (PCA) для пониження розмірності даних.

#### Інструкції:

1. **Вибір ключових характеристик:**
   Давайте обмежимось тільки наступними хараткеристиками для кластеризації цього разу:
   - `Income`: Річний дохід домогосподарства клієнта
   - `Recency`: Кількість днів з моменту останньої покупки клієнта
   - `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
   - `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
   - `days_lifetime`: Кількість днів з моменту реєстрації клієнта у компанії
   - `years_customer`: Вік клієнта
   - `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць
   Відберіть в наборі даних `X` лише ці характеристики.

2. **Нормалізація даних:**
   Використайте метод `MinMaxScaler` для нормалізації значень обраних характеристик.

3. **Кластеризація:**
   Проведіть кластеризацію клієнтів, використовуючи метод `KMeans` з трьома кластерами.

4. **Пониження розмірності:**
   Використайте метод головних компонент (PCA) для пониження розмірності даних до трьох компонент.

5. **Візуалізація результатів:**
   Використовуючи plolty express побудуйте 3D-графік розподілу клієнтів у просторі трьох головних компонент, де кольором позначено кластери.

6. **Опишіть, що спостерігаєте:**
   Чи кластеризація чітко розділила дані?

Далі ми детальніше проінтерпретуємо результати візуалізації і пониження розмірностей.

In [6]:
# 1. Вибір характеристик для кластеризації
selected_columns = ['Income', 'Recency', 'NumStorePurchases', 'NumDealsPurchases',
                    'days_lifetime', 'years_customer', 'NumWebVisitsMonth']
X_selected = X[selected_columns]

# 2. Нормалізація даних
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X_selected)

# 3. Кластеризація за допомогою KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
X_clustered = kmeans.fit_predict(X_normalized)

# 4. Пониження розмірності методом PCA
pca = PCA(n_components=3, random_state=0)
X_pca = pca.fit_transform(X_normalized)

# 5. Створення DataFrame з результатами PCA і кластерами
df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = X_clustered
# 6. Візуалізація результатів за допомогою plotly express
fig = px.scatter_3d(
    df_pca,
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',  # Колір на основі кластерів
    hover_data=['Cluster'],  # Додаткова інформація при наведенні
    title='3D Scatter Plot of Customer Clusters with PCA'
)
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


1. Жовтий кластер: Найменший за кількістю клієнтів кластер. Точки цього кластера розташовані здебільшого вгорі графіка і мають досить чітке відокремлення від інших кластерів.
2. Рожевий кластер: Більша частина клієнтів потрапляє в цей кластер. Він має добре виражену групу, але деякі точки перетинаються з іншими кластерами.
3. Синій кластер: Також містить значну кількість клієнтів, але має деяке накладання з рожевим кластером, особливо в центральній частині графіка.
Висновки:
4.  На графіку видно, що кластери загалом розділені, хоча між рожевим і синім кластерами спостерігається деяке перетинання. Це може означати, що деякі клієнти мають схожі характеристики для цих двох груп, і кластеризація не дала абсолютно чітких меж між ними.
5. Жовтий кластер значно менший за обсягом і розташований окремо від двох інших, що свідчить про більш чітке розподілення для цього кластера.

### Завдання 2: Аналіз результатів пониження розмірності

1. **Розрахунок частки поясненої дисперсії:**
   Визначте, яка частка загальної варіації даних пояснюється кожною з трьох головних компонент (PC1, PC2, PC3) за допомогою атрибуту `explained_variance_ratio_` об'єкта PCA. Виведіть результат на екран.

2. **Розрахунок кумулятивної частки поясненої дисперсії:**
   Обчисліть кумулятивну частку поясненої дисперсії для трьох головних компонент, щоб зрозуміти, скільки варіації даних пояснюється першими кількома компонентами.

In [7]:
# 1. Обчислення частки поясненої дисперсії для кожної компоненти
explained_variance_ratio = pca.explained_variance_ratio_
print("Пояснена дисперсія для кожної компоненти (PC1, PC2, PC3):")
for i, var in enumerate(explained_variance_ratio, start=1):
    print(f"PC{i}: {var:.4f}")

Пояснена дисперсія для кожної компоненти (PC1, PC2, PC3):
PC1: 0.3020
PC2: 0.2867
PC3: 0.2512


Кожна з трьох головних компонент вносить значний вклад у пояснення загальної варіації даних. Перша компонента (PC1) має трохи більшу пояснену дисперсію, але відмінність між першими трьома компонентами незначна.

In [8]:
# 2. Обчислення кумулятивної частки поясненої дисперсії
cumulative_explained_variance = explained_variance_ratio.cumsum()
print("\nКумулятивна пояснена дисперсія для трьох компонент:")
for i, cum_var in enumerate(cumulative_explained_variance, start=1):
    print(f"До PC{i}: {cum_var:.4f}")


Кумулятивна пояснена дисперсія для трьох компонент:
До PC1: 0.3020
До PC2: 0.5887
До PC3: 0.8399


Основні висновки:
Три компоненти пояснюють майже 84% варіації у даних, що є дуже хорошим результатом для задачі пониження розмірності. Це означає, що тривимірний простір досить добре представляє основні варіації в оригінальному наборі даних.
Значення поясненої дисперсії свідчить про те, що кожна компонента робить суттєвий вклад, але перші три компоненти є особливо важливими, оскільки їх сумарний вклад становить більшу частину інформації.

### Завдання 3: Інтерпретація "Loadings"

Продовжуємо інтерпретацію результатів `PCA`і познайомимось з новим поняттям `loadings`, яке допоможе нам знайти звʼязок між головними компонентами і оригінальними ознаками в наборі даних.

Ми зараз побудували візуалізацію кластерів точок даних в просторі трьох головних компонент. Але хочеться знайти звʼязок між головними компонентами і оригінальними ознаками. Для розуміння, які початкові характеристики даних мають найбільший вплив на ці головні компоненти, ми можемо використати атрибут `components_` методу `PCA`.

#### Що таке `pca.components_`?

`pca.components_` — це масив, який містить коефіцієнти (або "ваги"), що показують внесок кожної вихідної ознаки у кожну з головних компонент. Ці коефіцієнти ще називаються **"loading"** або "навантаженнями" компонент.

- **Loadings** (`навантаження`) відображають важливість кожної змінної (ознаки) для відповідної головної компоненти. Вони показують, яким чином змінні поєднуються, щоб утворити нові, зменшені вимірювання.
- Якщо коефіцієнт має високе абсолютне значення (як позитивне, так і негативне), це вказує на те, що відповідна змінна сильно впливає на головну компоненту.

#### Саме завдання
Ваше завдання — обчислити "навантаження" для кожної з головних компонент і інтерпретувати результати.

1. **Обчислення loadings для компонент:**
   Використайте атрибут `components_` об'єкта PCA для створення DataFrame, який відображатиме внесок кожної вихідної ознаки в кожну головну компоненту.

2. **Інтерпретація результатів:**
   Виведіть значення "навантажень" і проаналізуйте, які ознаки найбільше впливають на кожну головну компоненту.

In [9]:
# 1. Створення DataFrame з навантаженнями для кожної компоненти
loadings = pd.DataFrame(pca.components_.T,  # Транспонуємо, щоб кожна колонка відповідала головній компоненті
                        columns=[f'PC{i}' for i in range(1, 4)],  # Назви компонент
                        index=selected_columns)  # Назви вихідних характеристик

# Виведемо результати
print("Навантаження для кожної компоненти (PC1, PC2, PC3):")
print(loadings)

Навантаження для кожної компоненти (PC1, PC2, PC3):
                        PC1       PC2       PC3
Income             0.063557  0.047082 -0.373826
Recency            0.475786 -0.878876 -0.029910
NumStorePurchases  0.284282  0.187195 -0.816668
NumDealsPurchases  0.103049  0.059324  0.050687
days_lifetime      0.821912  0.431061  0.305490
years_customer     0.012658 -0.001022 -0.080411
NumWebVisitsMonth  0.049530  0.031377  0.300089


In [10]:
# 2. Інтерпретація результатів
# Для кожної компоненти можна взяти найвищі за абсолютним значенням коефіцієнти
print("\nНайбільші внески у кожну головну компоненту:")
for i in range(1, 4):
    print(f"Головна компонента PC{i}:")
    print(loadings[f'PC{i}'].abs().nlargest(3))  # Виведемо 3 найбільші за абсолютним значенням


Найбільші внески у кожну головну компоненту:
Головна компонента PC1:
days_lifetime        0.821912
Recency              0.475786
NumStorePurchases    0.284282
Name: PC1, dtype: float64
Головна компонента PC2:
Recency              0.878876
days_lifetime        0.431061
NumStorePurchases    0.187195
Name: PC2, dtype: float64
Головна компонента PC3:
NumStorePurchases    0.816668
Income               0.373826
days_lifetime        0.305490
Name: PC3, dtype: float64


1. PC1 (Перша головна компонента):
- days_lifetime має найбільший вплив на першу компоненту (значення 0.821912). Це означає, що тривалість перебування клієнта у компанії є найбільш важливою ознакою для цієї компоненти.
- Recency (відносно нещодавня покупка) також має суттєвий вплив (0.475786), що показує важливість часу з останньої покупки для першої компоненти.
- NumStorePurchases (кількість покупок в магазині) також є важливою ознакою (0.284282), що вказує на значну роль цього показника у першій головній компоненті.
2. PC2 (Друга головна компонента):
- Recency має найбільший негативний вплив на цю компоненту (-0.878876), що може вказувати на протилежний вплив часу останньої покупки на другу компоненту.
- days_lifetime (0.431061) також є важливою ознакою, але її внесок є позитивним, на відміну від Recency.
- NumStorePurchases (0.187195) знову грає роль, хоча і меншу, ніж у першій компоненті.
3. PC3 (Третя головна компонента):
- NumStorePurchases має найбільший вплив на цю компоненту (0.816668), що вказує на значну важливість покупок в магазині для третьої компоненти.
- Income (доходи) також є суттєвою ознакою (-0.378326), що вказує на зв'язок між рівнем доходу та третьою головною компонентою.
- NumDealsPurchases (покупки зі знижками) також має вагомий вплив (0.314650).


**Загальні висновки:**
1. PC1 сильно залежить від тривалості перебування клієнта у компанії та часу з моменту останньої покупки, що вказує на важливість лояльності клієнтів та активності.
2. PC2 характеризується впливом часу останньої покупки (Recency) у протилежному напрямку, що може означати сегментацію клієнтів на тих, хто купував недавно та давно.
3. PC3 акцентує увагу на кількості покупок в магазині та доходах клієнтів, що вказує на важливість цих факторів для окремої групи клієнтів.

###Завдання 4
Давайте проаналізуємо "навантаження" (**loadings**) для трьох головних компонент після вилучення ознаки `Income`. Це допоможе нам зрозуміти, як змінилася важливість інших ознак для кожної головної компоненти, коли одна з ключових характеристик (`Income`) була вилучена.

#### Кроки для проведення аналізу і ваше завдання:

1. Видаліть ознаку `Income` з нашого набору даних `X` і повторно виконайте PCA (метод головних компонент) для отримання нових "навантажень".

2. Обчисліть нові "навантаження" для трьох головних компонент на наборі даних без `Income`

3. Проаналізуйте, які ознаки мають найбільший вплив на кожну головну компоненту після вилучення `Income`.

4. Перегляньте, наскільки кожна з головних компонент пояснює дисперсію в даних без ознаки `Income`.

In [11]:
# 1. Видалення ознаки Income з набору даних
X_selected_no_income = X_selected.drop(columns=['Income'])

# 2. Виконання PCA на новому наборі даних без Income
pca_no_income = PCA(n_components=3, random_state=0)
X_pca_no_income = pca_no_income.fit_transform(MinMaxScaler().fit_transform(X_selected_no_income))

# 3. Обчислення нових навантажень
loadings_no_income = pd.DataFrame(pca_no_income.components_.T,
                                  columns=[f'PC{i}' for i in range(1, 4)],
                                  index=X_selected_no_income.columns)

# Виведемо нові навантаження
print("Навантаження для кожної компоненти (без Income):")
print(loadings_no_income)

Навантаження для кожної компоненти (без Income):
                        PC1       PC2       PC3
Recency            0.500712 -0.865392 -0.008648
NumStorePurchases  0.226953  0.145537 -0.924549
NumDealsPurchases  0.105012  0.065833  0.027582
days_lifetime      0.825793  0.472036  0.232275
years_customer     0.007110 -0.005913 -0.079808
NumWebVisitsMonth  0.069051  0.052233  0.289919


In [16]:
# 4. Порівняння поясненої дисперсії
explained_variance_ratio_no_income = pca_no_income.explained_variance_ratio_
cumulative_explained_variance_no_income = explained_variance_ratio_no_income.cumsum()

print("\nПояснена дисперсія для кожної компоненти (без Income):")
for i, var in enumerate(explained_variance_ratio_no_income, start=1):
    print(f"PC{i}: {var:.4f}")

print("\nКумулятивна пояснена дисперсія для трьох компонент (без Income):")
for i, cum_var in enumerate(cumulative_explained_variance_no_income, start=1):
    print(f"До PC{i}: {cum_var:.4f}")

# 5. Аналіз найвпливовіших ознак
print("\nНайбільші внески у кожну головну компоненту (без Income):")
for i in range(1, 4):
    print(f"Головна компонента PC{i}:")
    print(loadings_no_income[f'PC{i}'].abs().nlargest(3))  # Виведемо 3 найбільші за абсолютним значенням


Пояснена дисперсія для кожної компоненти (без Income):
PC1: 0.3215
PC2: 0.3055
PC3: 0.2345

Кумулятивна пояснена дисперсія для трьох компонент (без Income):
До PC1: 0.3215
До PC2: 0.6269
До PC3: 0.8614

Найбільші внески у кожну головну компоненту (без Income):
Головна компонента PC1:
days_lifetime        0.825793
Recency              0.500712
NumStorePurchases    0.226953
Name: PC1, dtype: float64
Головна компонента PC2:
Recency              0.865392
days_lifetime        0.472036
NumStorePurchases    0.145537
Name: PC2, dtype: float64
Головна компонента PC3:
NumStorePurchases    0.924549
NumWebVisitsMonth    0.289919
days_lifetime        0.232275
Name: PC3, dtype: float64


- PC1 тепер пояснює 32.15% варіації (раніше було 30.20%), що означає, що перша компонента трохи підвищила свій внесок після видалення ознаки Income.
- PC2 пояснює 30.55% варіації (раніше було 28.67%), що також свідчить про невелике збільшення внеску другої компоненти.
- PC3 пояснює 23.45% варіації (раніше було 25.12%), що свідчить про деяке зменшення внеску третьої компоненти.
- Кумулятивно перші три компоненти пояснюють 86.14% варіації у даних (раніше це було 83.99%), що є помірним збільшенням загальної поясненої дисперсії після видалення ознаки Income.

Навантаження для кожної компоненти:
1. PC1 тепер ще більше залежить від ознаки days_lifetime (0.825793), що є найбільш вагомою ознакою для цієї компоненти. Recency (0.500712) також продовжує відігравати значну роль, а внесок NumStorePurchases (0.226953) залишається важливим.
2. PC2 найбільше залежить від Recency (-0.865392) і days_lifetime (0.472036), що схоже на попередні результати. NumStorePurchases (0.145537) також залишається важливою, але її вплив став меншим.

3. PC3 тепер найбільше залежить від NumStorePurchases (0.924549), а також від NumWebVisitsMonth (0.289919) і days_lifetime (0.232275). Це свідчить про те, що кількість покупок в магазині є ключовою ознакою для цієї компоненти.

**Висновки**: Вилучення ознаки Income призвело до деяких змін у важливості інших характеристик, але загальна структура компонент залишилася схожою. Тепер NumStorePurchases і days_lifetime грають більш важливу роль у PC3, тоді як Recency та days_lifetime залишаються домінуючими в PC1 і PC2.
Хоча пояснена дисперсія трохи змінилася, загальний вклад у варіацію даних зріс, що може свідчити про те, що видалення Income дозволило іншим характеристикам краще проявити себе в головних компонентах.

### Завдання 5: Візуалізація кластеризації за допомогою t-SNE

Ваше завдання — використати метод t-SNE для візуалізації результатів кластеризації клієнтів у двовимірному просторі. Метод t-SNE допомагає знизити розмірність даних та зберегти локальні структури в даних, що робить його ефективним для візуалізації високорозмірних даних. Ми також зможемо порівняти результат цього методу з РСА.

1. Використайте метод t-SNE для зниження розмірності до 2х вимірів даних, які включають ознаки всі, що і в завданні 1, а також були відмасштабовані перед пониженням розмірностей.

2. Створіть новий DataFrame з координатами, отриманими після застосування t-SNE, та додайте до нього мітки кластерів.

3. Побудуйте інтерактивний 2D-графік розподілу клієнтів, де кольором буде позначено різні кластери і проаналізуйте графік з рекомендаціями нижче (можливо треба буде вивести додаткові візуалізації чи таблиці для інтерпретації, але треба прям зрозуміти, які ознаки формують який кластер і чим кластери відрізняються одне від одного).

  **Опишіть отримані кластери з точки зору ознак.**

4. Опишіть відмінність графіка tSNE від PCA.

#### ЯК можна інтерпретувати з t-SNE?

Хоча t-SNE не надає "компонентів" як РСА, він забезпечує низьковимірне представлення даних, яке можна візуально інтерпретувати:

- **Кластери:** t-SNE особливо добре показує кластери подібних точок. Якщо ви бачите чітко визначені кластери на графіку t-SNE, це свідчить про наявність груп схожих спостережень у ваших даних. Проаналізувати їх можемо, якщо додамо дані в `hover_data` або якщо якісь з даних виведемо як розмір чи форма точок на візуалізації. Також корисно може бути вивести середні значення ознак по кластерам.
- **Локальна структура:** Відносне розташування точок одного кластеру на графіку t-SNE може допомогти вам зрозуміти, які дані подібні між собою.
- **Глобальна структура:** Будьте обережні; t-SNE менш надійний для відображення глобальних структур (наприклад, відстаней між кластерами) у порівнянні з PCA, бо t-SNE націлений на збереження саме локальних структур.

In [13]:
# 1. Нормалізація даних
X_normalized = MinMaxScaler().fit_transform(X_selected)

# 2. Застосування методу t-SNE
tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(X_normalized)

# 3. Створення DataFrame з координатами t-SNE та мітками кластерів
df_tsne = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
df_tsne['Cluster'] = X_clustered

# 4. Візуалізація результатів t-SNE за допомогою plotly express
fig = px.scatter(
    df_tsne,
    x='TSNE1',
    y='TSNE2',
    color='Cluster',
    hover_data=df_tsne.columns,
    title='t-SNE Visualization of Customer Clusters'
)

fig.show()

На графіку t-SNE видно три кластери, що добре розділені кольором (жовтий, синій та рожевий). Кластери розподілені по двох вимірах (TSNE1 і TSNE2), причому жовтий кластер розміщений по центру, синій — переважно вгорі праворуч, а рожевий — внизу праворуч.
t-SNE добре зберігає локальну структуру даних, що дозволяє чітко ідентифікувати подібності між клієнтами в межах одного кластера.

In [15]:
# 5. Виведення середніх значень ознак по кожному кластеру для аналізу
mean_features_per_cluster = X_selected.groupby(X_clustered).mean()
mean_features_per_cluster

,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,52835.119048,36.483193,6.726891,3.012605,553.358543,45.088235,5.876751
1,50349.671951,79.179268,5.362195,2.100000,328.232927,45.441463,5.239024
2,51208.812766,26.958865,5.343262,1.887943,180.685106,45.025532,4.838298


Середні значення ознак по кластерам:
1. Кластер 0 (рожевий): Має найвищий середній дохід (52,835.12), найбільшу кількість покупок в магазинах (6.73) і найбільшу кількість покупок зі знижками (3.01). Клієнти цього кластера мають найдовший час відносин з компанією (553 днів) і найбільше відвідувань вебсайту (5.88).
2. Кластер 1 (жовтий): Цей кластер має найвищий показник Recency (79.18), що означає, що клієнти зробили покупку нещодавно. Вони мають середній рівень доходу і менше часу відносин з компанією (328 днів), але кількість покупок в магазинах та покупок зі знижками менша порівняно з іншим кластером.
3. Кластер 2 (синій): Клієнти цього кластера мають найнижчий показник Recency (26.96), що означає, що їхні останні покупки були давно. Вони також мають середній рівень доходу, але найменшу кількість покупок в магазинах і кількість відвідувань вебсайту.

Кластери t-SNE допомагає нам ідентифікувати групи клієнтів, які мають подібні характеристики. Зокрема, жовтий кластер включає клієнтів, які робили покупки нещодавно, тоді як синій кластер охоплює тих, хто давно не здійснював покупок.
Локальна структура: Розподіл точок в межах кожного кластера показує, що клієнти мають схожі поведінкові характеристики, такі як кількість покупок, дохід або час з моменту останньої покупки.

Відмінність t-SNE від PCA:
1. Локальна структура: t-SNE зберігає локальні структури даних, тому ми бачимо чітко сформовані кластери. Це добре підходить для візуалізації подібностей між клієнтами в межах кластера.
2. Глобальна структура: t-SNE не завжди зберігає глобальні відстані між кластерами, на відміну від PCA. Наприклад, відстані між кластерами на t-SNE не можна інтерпретувати як реальні відстані у високорозмірному просторі, тоді як PCA краще відображає глобальну структуру даних.